So first, let's take a look at how a block cipher works. This is contrived, in that we're using ciphers that don't require blocking, but bear with me. This illustrates the concept. We're going to start with a string, then block that string into groups of five characters, encrypt those blocks, and concatenate them.

In [2]:
import simple_ciphers as sc

def apply_cipher(cipher, plaintext):
    # Make upper case and remove spaces prior to slicing
    plaintext = plaintext.upper().strip().replace(' ', '')
    
    # Make slices in groups of 5 or less
    slices = [plaintext[i:i+5] for i in range(0, len(plaintext), 5)]
    
    # Pad any slices less than five characters long
    slices = [slice if len(slice) == 5 else slice + 'X' * (5 - len(slice)) for slice in slices]

    # encipher the processed slices
    enciphered_slices = [cipher.encipher(slice) for slice in slices]
    
    # Let's see what we have!
    print('<=== ({})'.format(type(cipher)))
    print('Blocked ciphertext\t {}'.format(''.join(enciphered_slices)))
    print('Non-blocked ciphertext\t {}'.format(cipher.encipher(plaintext)))
    print('===>')

plaintext = 'This is a sooper sekkrit message'

vigenere = sc.Vigenere('iskey')
apply_cipher(cipher=vigenere, plaintext=plaintext)

bifid = sc.Bifid()
apply_cipher(cipher=bifid, plaintext=plaintext)


<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOWHBV
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHDXGS
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>


Well, this doesn't do much for us, does it? I mean, the blocked and non-blocked ciphertext are the same. Let's introduce something new - a string that we'll prepend to each block. We'll call this an initialization vector, or an IV for short. Let's see what this does.

This is called __Electronic Codebook Mode__, and is a mode in which we can use block ciphers. It's pretty simple, but not very effective, even when using more complex ciphers. A big problem with ECB is enciphering the same string with the same key. This will yield the same ciphertext, and this really helps crypanalysts, so you want to avoid it in ciphers you develop.

In [3]:
def apply_cipher(cipher, plaintext, iv=''):
    plaintext = plaintext.upper().strip().replace(' ', '')
    slices = [plaintext[i:i+5] for i in range(0, len(plaintext), 5)]
    slices = [slice if len(slice) == 5 else slice + 'X' * (5 - len(slice)) for slice in slices]
    
    # Add the IV here!
    enciphered_slices = [cipher.encipher(iv + slice) for slice in slices]
    print('<=== ({})'.format(type(cipher)))
    print('Blocked ciphertext\t {}'.format(''.join(enciphered_slices)))
    print('Non-blocked ciphertext\t {}'.format(cipher.encipher(plaintext)))
    print('===>')

iv = 'ab'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)

iv = 'cd'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)


<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 ITDLGAAITCEQWGITZIPAWITUOPQLITWIQASITQIVFP
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 NWBLNZZNCCLLDPNHCEPCRNDCNWITNHCNGCCNHFLGDS
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>
<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 KVDLGAAKVCEQWGKVZIPAWKVUOPQLKVWIQASKVQIVFP
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 VWBXNZZVCCXLDPVHCOPCRVDCKWITVHCKGCCVHFXGDS
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>


Well, this is interesting. Using the IV pretty radically changed the ciphertext. It not only made it longer, but it also changed the letters. So, in this case, we took the blocks of five after having been padded and appended the initialization vector to the beginning of each block. Then we concatenated the blocks, so each block was seven characters long - two characters for the initialization vector and five for the plaintext block.

Now, this is still pretty weak, and isn't a recognized cipher block mode, but it does lead is in the right direction. Let's try something else with the IV.

In [4]:
def apply_cipher(cipher, plaintext, iv=''):
    plaintext = plaintext.upper().strip().replace(' ', '')
    
    
    slices = [plaintext[i:i+5] for i in range(0, len(plaintext), 5)]
    slices = [slice if len(slice) == 5 else slice + 'X' * (5 - len(slice)) for slice in slices]
    
    enciphered_slices = []
    for i, slice in enumerate(slices):
        # Convert the counter to a letter, starting from A
        cntr = chr(i + 65)
        
        # Prepend the IV to the counter, pad the counter so the 
        # combined string is the same length as a block
        iv_cntr = iv.upper() + 'X' * (5 - (len(cntr) + len(iv))) + cntr
        
        # Encipher the IV|CNTR string
        cipher_cntr = cipher.encipher(iv_cntr)
        
        # Encpher the plaintext block with the ciphered IV|CNTR string
        # as the key. We're using the Vigenere cipher here, but
        # in binary this is an XOR operation.
        ciphered_slice = sc.Vigenere(cipher_cntr).encipher(slice)
        
        # Append the newly enciphered slice
        enciphered_slices.append(ciphered_slice)
    
    
    #enciphered_slices = [cipher.encipher(iv + slice) for slice in slices]
    print('<=== ({})'.format(type(cipher)))
    print('Blocked ciphertext\t {}'.format(''.join(enciphered_slices)))
    print('Non-blocked ciphertext\t {}'.format(cipher.encipher(plaintext)))
    print('===>')

iv = 'ab'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)

iv = 'cd'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)


<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 BAPTGATZPNXXYTESDYJUUXZTCOXEYA
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 GKIDKFDTZQCHTDGXNWTOZHSDRTHCIZ
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>
<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 DCPTGCVZPNZZYTEUFYJUWZZTCQZEYA
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 OKIPKNDTLQKHTPGFNWFOHHSPRBHCUZ
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>


So what did we do?

We took the IV and prepended it to a counter. Then we encrypted the IV|CNTR string, and we took that string, and used it as a key to a Vigenere cipher to encrypt a block of plaintext. We used this ciphertext to create the enciphered message. So for each block $i$, this is the process we used:

$ IV|N_i \longrightarrow C_i' = E_K(IV|N_i) \longrightarrow C_i = E_{C_i'}'(P) \longrightarrow C_i$

This is essentially __Counter Mode__, another encryption mode for block ciphers. CTR is officially:

$ IV|N_i \longrightarrow C_i' = E_K(IV|N_i) \longrightarrow C_i = C_i' \oplus P_i \longrightarrow C_i$

Described more tersely, CTR is:

$C_i = E_k(IV|N_i) \oplus P_i$

So each encrypted block $C_i$ results from the encipherment if the encrypted initialization vector prepended to the counter $IV|N_i$ XOR the plaintext block $P_i$. In the example I used, I didn't use XOR because I'm not using binary data - the example is just illustrative. Nevertheless, modern encryption encrypts over binary strings, and would use XOR.

What else can we do?

In [6]:
def apply_cipher(cipher, plaintext, iv=''):
    plaintext = plaintext.upper().strip().replace(' ', '')
    
    
    slices = [plaintext[i:i+5] for i in range(0, len(plaintext), 5)]
    slices = [slice if len(slice) == 5 else slice + 'X' * (5 - len(slice)) for slice in slices]
    
    c_1 = cipher.encipher(sc.Vigenere(iv.upper()).encipher(slices[0]))
    slices = slices[1:len(slices)]
    enciphered_slices = []
    enciphered_slices.append(c_1)
    enciphered_slice = c_1
    for i, slice in enumerate(slices):
        # Encipher a new slice using the ciphertext of the previous operation
        # as the key to the embedded cipher
        enciphered_slice = cipher.encipher(sc.Vigenere(enciphered_slice).encipher(slice))
        
        # Append the newly enciphered slice
        enciphered_slices.append(enciphered_slice)
    
    
    #enciphered_slices = [cipher.encipher(iv + slice) for slice in slices]
    print('<=== ({})'.format(type(cipher)))
    print('Blocked ciphertext\t {}'.format(''.join(enciphered_slices)))
    print('Non-blocked ciphertext\t {}'.format(cipher.encipher(plaintext)))
    print('===>')

iv = 'ab'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)

iv = 'cd'
apply_cipher(cipher=vigenere, plaintext=plaintext, iv=iv)
apply_cipher(cipher=bifid, plaintext=plaintext, iv=iv)


<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 BASXGBSUPSYOVLUQQWXLKMYTJYIFUE
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 IITIZNNATMSCHPVRYFURXOVLWXZUST
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>
<=== (<class 'simple_ciphers.Vigenere'>)
Blocked ciphertext	 DCUZIDUWRUAQXNWSSYZNMOAVLAKHWG
Non-blocked ciphertext	 BZSWGASCSMXWBWCSCBMRUWCWYOW
===>
<=== (<class 'simple_ciphers.Bifid'>)
Blocked ciphertext	 VXDIKYZNVQYAIOUUZNIMQFGFNIXOUN
Non-blocked ciphertext	 WTTNZCVXLPHSEPRDZIWTHSNGCHR
===>


Here, we've done something a little different. We use the ciphertext from the previous calculation as a key to encipher the next block. The drawback to this approach is that it then requires each calculation to happen serially. So, the above operation looks like this:

$ i = 1 : IV \longrightarrow C_1 = E_K(E_{IV}(P_1)) \longrightarrow C_1$; 

$ i > 1 : C_1 \longrightarrow C_i = E_K(E_{C_{i-1}}(P_i)) \longrightarrow C_i$

This is essentially __Cipher Block Chaining__ mode, another encryption mode for block ciphers. CBC is officially:

$ i = 1 : C_1 = E_K(P_1 \oplus IV) $; 

$ i > 1 : C_i = E_K(P_i \oplus C_{i-1}) $

These are two block modes. There are two others supported defined by NIST for use with block ciphers, including __Cipher Feedback Mode__ and __Output Feedback Mode__. They act in similar ways. The point of these manipulations is to avoid using the same key over the same sequence of symbols (letters or bits). In either case, if you use the same key over the same information, the cipher text will be the same. This greatly helps cryptanalysis. You __never__ use the same initialization vector with the same plaintext. Generally, you want to avoid using the same initialization vector period.
